In [1]:
import torch 

%cd ../


/workspace/Walk_Video_PyTorch


In [2]:
from  project.models.pytorchvideo_models import WalkVideoClassificationLightningModule

/usr/lib/python3.9/site-packages/apex/pyprof/__init__.py:5: FutureWarning: pyprof will be removed by the end of June, 2022
  warnings.warn("pyprof will be removed by the end of June, 2022", FutureWarning)


In [3]:

model = WalkVideoClassificationLightningModule.load_from_checkpoint("/workspace/Walk_Video_PyTorch/lightning_logs/lightning_logs/resnet/checkpoints/epoch=44-step=765.ckpt", model_type="resnet")



In [4]:
class opt:
        _DATA_PATH = "/workspace/data/dataset/predict/"
        _CLIP_DURATION = 2 # Duration of sampled clip for each video
        _BATCH_SIZE = 8
        _NUM_WORKERS = 0  # Number of parallel processes fetching data
        model_type = 'resnet'

In [5]:
from project.dataloader.data_loader import WalkDataModule

pred_data = WalkDataModule(opt).pred_dataloader()

input_data = next(iter(pred_data))

In [6]:
model = model.eval()

In [7]:
input_data['video'].shape

input_data["video_name"]

['ca8.mp4',
 'c5.mp4',
 'ca9.mp4',
 'c2.mp4',
 'ca1.mp4',
 'c8.mp4',
 'lcs_full_lat.MP4',
 'ca3.mp4']

In [8]:
input_data['label']

tensor([1, 0, 1, 0, 1, 0, 1, 1])

In [9]:
preds = model(input_data['video'])

preds

tensor([[ 22.9373, -23.1229],
        [ 22.7133, -22.9079],
        [ -1.5771,   1.5693],
        [ 19.5539, -19.7164],
        [ 18.1080, -18.2500],
        [  9.0107,  -9.1020],
        [ 22.5791, -22.8260],
        [ 16.7603, -16.9040]], grad_fn=<ViewBackward0>)

In [40]:
post_act = torch.nn.Softmax(dim=1)
preds = post_act(preds)
pred_classes = preds.topk(k=2).indices

In [43]:
preds.topk(k=2).values.tolist()

[[0.5070546269416809, 0.49294543266296387],
 [0.5070546269416809, 0.49294543266296387],
 [0.5065709352493286, 0.493429034948349],
 [0.5070546269416809, 0.49294543266296387],
 [0.5070546269416809, 0.49294543266296387],
 [0.5070546269416809, 0.49294543266296387],
 [0.5070546269416809, 0.49294543266296387],
 [0.5070546269416809, 0.49294543266296387]]

In [30]:
preds

tensor([[0.5282, 0.4718],
        [0.5282, 0.4718],
        [0.4737, 0.5263],
        [0.5282, 0.4718],
        [0.5282, 0.4718],
        [0.5282, 0.4718],
        [0.5282, 0.4718],
        [0.5282, 0.4718]], grad_fn=<SoftmaxBackward0>)

In [38]:
pred_classes

tensor([[0, 1],
        [0, 1],
        [1, 0],
        [0, 1],
        [0, 1],
        [0, 1],
        [0, 1],
        [0, 1]])

In [13]:
classname = {}

classname[0] = 'asd'
classname[1] = 'lcs'

In [14]:
real_calss = []

for i in input_data['label'].tolist():
    real_calss.append(classname[i])

In [33]:
real_calss

['lcs', 'asd', 'lcs', 'asd', 'lcs', 'asd', 'lcs', 'lcs']

In [39]:
# pred_class_names = []
# for num in range(opt._BATCH_SIZE):
#     for i in pred_classes[i]:
#         pred_class_names.append(classname[int(i)])


pred_class_names = [classname[int(i)] for i in pred_classes[0]]
print("Predicted labels: %s" % ", ".join(pred_class_names))
print("real label: %s" % ",".join(real_calss))

Predicted labels: asd, lcs
real label: lcs,asd,lcs,asd,lcs,asd,lcs,lcs
